# Cargar librerías

In [1]:
from firebase import firebase
import json
from transformers import AutoTokenizer, BertForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
from datasets import Dataset, load_metric
import numpy as np
import re
import dotenv
import os

/Users/javiervilarinomayo/opt/anaconda3/envs/ppi/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Lectura e integración de datos

El conjunto de datos usado durante la realización de este trabajo es privado y no puede ser compartido, pero si se puede compartir el código de la lectura y la integración de los datos. El conjunto de datos final está formado por 3088 definiciones de indicadores de rendimiento de procesos (PPIs), donde cada indicador, además de incluir la definición, incluye una etiqueta asociada a cada palabra de la definición. 

El objetivo de este proyecto, es predecir la etiqueta asociada a cada palabra de la definición de un indicador de rendimiento de procesos (PPI). Esta etiquetación tiene una gran utilidad, ya que, si se realiza adecuadamente, se puede computar posteriormente el indicador de proceso.

### Datos iniciales

En primer lugar, realizamos la lectura del conjunto de datos inicial. Cabe destacar, que este conjunto de datos es bastante limitado, y que por tanto será necesario realizar una ampliación de los mismos a través de distintas técnicas.

In [37]:
with open('./data/parser_training_data.json', 'r') as f:
    training_data = json.load(f)

### Datos de crowdsourcing

La primera opción que planteamos para solventar el problema del tamaño inicial del conjunto de datos, fue la de realizar una ampliación de estos generando parafrases que posteriormente serían etiquedas por usuarios reales a través de una plataforma de crowdsourcing.

El resultado de este proceso supuso una ampliación de los datos que resultó ser insuficiente para solventar el problema de tamaño del conjunto de datos.

In [38]:
dotenv.load_dotenv()
FIREBASE_URL = os.getenv("FIREBASE_URL")    

firebase_url = firebase.FirebaseApplication(FIREBASE_URL, None)

best_paraphrases_firebase = firebase_url.get('/best_paraphrases', None)
firebase_id = list(best_paraphrases_firebase)[0]
best_paraphrases = best_paraphrases_firebase[firebase_id]

paraphrases = firebase_url.get('/paraphrases', None)
firebase_paraphrase_id = list(paraphrases)[0]
tagged_paraphrases = paraphrases[firebase_paraphrase_id]

### Datos generados con chatito

Con el objetivo de generar el mayor número de datos posible, se planteó la posibilidad de generar datos a través de la herramienta chatito. Esta herramienta permite realizar esta generación a partir de un fichero de definición de patrones.

In [39]:
with open('./data/training_chatito.json', 'r') as f:
    training_chatito = json.load(f)["measures"]

# Preprocesamiento de datos

En el caso del conjunto de datos generado a través de la plataforma de crowdsourcing, es necesario realizar un preprocesamiento específico debido a la forma en la que los datos se almacenan en firebase y a las decisiones de diseño tomadas durante la implementación de la plataforma. 

En este caso diferenciamos entre parafrases etiquetadas que han sido confirmadas por un número relevante de usuarios y aquellas que no han sido totalmente confirmadas. Cabe destacar que debido a la escasa cantidad de datos, se ha decidido incluir todas las parafrases etiquetadas, independientemente de si han sido confirmadas o no.

In [40]:
def get_best_annotation(paraphrase, tagged_paraphrases):
    same_paraphrases = [p["annotation"] for p in tagged_paraphrases if p["description"] == paraphrase]
    number_occurrences = [same_paraphrases.count(a) for a in same_paraphrases]
    max_occurrences = max(number_occurrences)
    return same_paraphrases[number_occurrences.index(max_occurrences)]

In [41]:
paraphrases = set([phrase["description"] for phrase in tagged_paraphrases["data"]])
for paraphrase in paraphrases:
    annotation = get_best_annotation(paraphrase, tagged_paraphrases["data"])
    best_paraphrases["data"].append({"description": paraphrase, "annotation": annotation})

Respecto al conjunto de datos generado a través de chatito, se realiza un preprocesamiento básico, donde se realiza una conversión de formato de los datos para que estén
 en el mismo formato que el conjunto de datos inicial.

In [42]:
training_chatito_parsed = []
for phrase in training_chatito:
    description = "".join([value["value"] for value in phrase])
    slots = [{'text': value["value"].strip(), 'tag': 'O'} if "slot" not in value else {'text': value["value"].strip(), 'tag': value["slot"]} for value in phrase]
    slots_cleaned = [slot for slot in slots if slot["text"] != ""]
    slots_list = []
    for slot in slots_cleaned:
        slots_list.append({'text': slot["text"], 'tag': slot["tag"]})
    training_chatito_parsed.append({"description": description, "annotation": slots_list})

### Integración de datos

Una vez están los datos preparados, se procede a su integración. En este caso, se realiza una integración de los datos de forma que se mantengan los datos originales y se añadan los nuevos datos.

In [43]:
best_paraphrases["data"].extend(training_data["data"])
best_paraphrases["data"].extend(training_chatito_parsed)

In [44]:
print(f'Number of phrases: {len(best_paraphrases["data"])}')

Number of phrases: 3190


A continuación, decidimos centrar el alcance del proyecto a PPIs relacionados con métricas temporales. Para ello, se realiza un filtrado de los datos para quedarnos únicamente con aquellos PPIs que contienen etiquetas relacionadas con métricas temporales.

### Filtrado de datos

In [45]:
TIME_TAGS = ["TMI", "TSI", "TSE", "TEI", "TEE", "TBE"]
COUNT_TAGS = ["CMI", "CE"]
DATA_TAGS = ["AttributeName, AttributeValue"]

In [46]:
time_phrases = []
count_phrases = []
data_phrases = []

for phrase in best_paraphrases["data"]:
    text = phrase["description"]
    labels = set([label["tag"] for label in phrase["annotation"]])
    if len(labels.intersection(TIME_TAGS)) > 0:
        time_phrases.append(phrase)
    elif len(labels.intersection(COUNT_TAGS)) > 0:
        count_phrases.append(phrase)
    elif len(labels.intersection(DATA_TAGS)) > 0:
        data_phrases.append(phrase)

print(f'Number of time phrases: {len(time_phrases)}')
print(f'Number of count phrases: {len(count_phrases)}')
print(f'Number of data phrases: {len(data_phrases)}')

Number of time phrases: 3088
Number of count phrases: 81
Number of data phrases: 0


### Reajuste del formato

En los siguientes pasos, se formatean los datos para que se ajusten a los requerimientos de la libreria datasets.

In [47]:
data = []
useless_tags = ["TMI", "TSI", "TEI", "GBI"]

for phrase in time_phrases:
    annotations = []
    for slot in phrase["annotation"]:
        slot_object = {}
        slot_object["value"] = slot["text"]
        slot_object["type"] = "Slot"
        slot_object["slot"] = slot["tag"] if slot["tag"] not in useless_tags else "O"
        annotations.append(slot_object)

    data.append(annotations)

In [48]:
tokens = []
tags = []

for phrase in data:
    phrase_tokens = []
    phrase_tags = []
    for slot in phrase:
        splits = slot["value"].split(" ")
        tag = slot["slot"]
        for i in range(len(splits)):
            if tag != "O":
                if i == 0:
                    phrase_tokens.append(splits[i])
                    phrase_tags.append("B-"+tag)
                else:
                    phrase_tokens.append(splits[i])
                    phrase_tags.append("I-"+tag)
            else:
                phrase_tokens.append(splits[i])
                phrase_tags.append(tag)
    tokens.append(phrase_tokens)
    tags.append(phrase_tags)

Una vez procesados los datos, se procede a la instanciación de la clase Dataset. Además, generamos la lista de etiquetas codificadas.

In [49]:
tags_list = list(set([tag for phrase in tags for tag in phrase]))
print(tags_list)

labels = [[tags_list.index(label) for label in phrase] for phrase in tags]

examples = {
    "tokens": tokens,
    "tags": labels
}

datasets = Dataset.from_dict(examples).train_test_split(test_size=0.2)

['B-TBE', 'I-AttributeValue', 'I-GBC', 'I-AGR', 'B-TEE', 'B-CCI', 'B-AGR', 'I-TSE', 'I-TEE', 'B-GBC', 'B-AttributeValue', 'B-TSE', 'I-TBE', 'O', 'I-CCI']


# Tokenizer

Una vez realizado el preprocesamiento de los datos, antes de realizar el fine-tuning del modelo, es necesario realizar un tokenizado de los datos. Para ello, se utiliza la clase AutoTokenizer de la librería transformers. Cabe destacar, que se usa el tokenizador de bert-base-uncased, ya que es el modelo que se usará para realizar el fine-tuning.

In [50]:
model_checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
batch_size = 64

loading configuration file config.json from cache at /Users/javiervilarinomayo/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.txt from cache at /Users/javiervilarinomayo/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b

In [51]:
label_all_tokens = True

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [52]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

100%|██████████| 1/1 [00:00<00:00, 25.48ba/s]


# Fine-tuning

Una vez tokenizados los datos, podemos proceder a preparar el fine-tuning del modelo. Para ello, se utiliza la clase BertForTokenClassification de la librería transformers para cargar el modelo preentrenado. En este punto, configuramos ciertos parámetros del modelo, como el número de epochs, el tamaño del batch, el tamaño del learning rate, etc.

In [53]:
model = BertForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(tags_list))

loading configuration file config.json from cache at /Users/javiervilarinomayo/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_12": 12,
    "LABEL_13": 13,
    "LABEL_14": 14,
    "LABEL_2": 

In [54]:
args = TrainingArguments(
    "TimeClassification",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    optim="adamw_torch",
    push_to_hub=False,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Instanciamos DataCollatorForTokenClassification, que realizará el padding dinámico de los datos, para que todos los datos tengan el mismo tamaño.

In [55]:
data_collator = DataCollatorForTokenClassification(tokenizer)

Además, cargamos la métrica seqeval que nos permitirá evaluar el modelo a través de distintas métricas.

In [56]:
metric = load_metric("seqeval")

In [57]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [tags_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [tags_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Una vez está todo listo, procedemos a realizar el fine-tuning del modelo.

In [58]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [59]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tags, tokens. If tags, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 2470
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 117
  Number of trainable parameters = 108903183
 33%|███▎      | 39/117 [04:08<07:07,  5.48s/it]The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tags, tokens. If tags, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 618
  Batch size = 64
                                                
 33%

{'eval_loss': 0.6913461089134216, 'eval_precision': 0.5617005267118134, 'eval_recall': 0.6528202885876694, 'eval_f1': 0.6038422649140545, 'eval_accuracy': 0.8204946160217725, 'eval_runtime': 21.1813, 'eval_samples_per_second': 29.177, 'eval_steps_per_second': 0.472, 'epoch': 1.0}


 67%|██████▋   | 78/117 [08:38<03:41,  5.67s/it]The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tags, tokens. If tags, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 618
  Batch size = 64
                                                
 67%|██████▋   | 78/117 [08:59<03:41,  5.67s/it]

{'eval_loss': 0.1956333965063095, 'eval_precision': 0.8754654530409599, 'eval_recall': 0.9252295583734149, 'eval_f1': 0.8996598639455782, 'eval_accuracy': 0.9604780499349189, 'eval_runtime': 21.4215, 'eval_samples_per_second': 28.85, 'eval_steps_per_second': 0.467, 'epoch': 2.0}


100%|██████████| 117/117 [13:56<00:00,  7.45s/it]The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tags, tokens. If tags, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 618
  Batch size = 64
                                                 
100%|██████████| 117/117 [14:18<00:00,  7.45s/it]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 117/117 [14:18<00:00,  7.45s/it]

{'eval_loss': 0.11802513152360916, 'eval_precision': 0.9482242190842961, 'eval_recall': 0.9689549628334062, 'eval_f1': 0.9584775086505191, 'eval_accuracy': 0.9814223168855757, 'eval_runtime': 21.2059, 'eval_samples_per_second': 29.143, 'eval_steps_per_second': 0.472, 'epoch': 3.0}
{'train_runtime': 858.1347, 'train_samples_per_second': 8.635, 'train_steps_per_second': 0.136, 'train_loss': 0.7391044421073718, 'epoch': 3.0}


100%|██████████| 117/117 [14:19<00:00,  7.35s/it]


TrainOutput(global_step=117, training_loss=0.7391044421073718, metrics={'train_runtime': 858.1347, 'train_samples_per_second': 8.635, 'train_steps_per_second': 0.136, 'train_loss': 0.7391044421073718, 'epoch': 3.0})

Con el siguiente código, podemos visualizar el resultado del fine-tuning, obteniendo el accuracy, el f1-score, recall y precision.

In [61]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tags, tokens. If tags, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 618
  Batch size = 64
100%|██████████| 10/10 [00:20<00:00,  2.07s/it]


{'eval_loss': 0.11802513152360916,
 'eval_precision': 0.9482242190842961,
 'eval_recall': 0.9689549628334062,
 'eval_f1': 0.9584775086505191,
 'eval_accuracy': 0.9814223168855757,
 'eval_runtime': 22.6685,
 'eval_samples_per_second': 27.263,
 'eval_steps_per_second': 0.441,
 'epoch': 3.0}

Además, una vez entrenado y evaluado el modelo, podemos guardar el modelo para poder utilizarlo posteriormente.

In [62]:
trainer.save_model()

Saving model checkpoint to TimeClassification
Configuration saved in TimeClassification/config.json
Model weights saved in TimeClassification/pytorch_model.bin
tokenizer config file saved in TimeClassification/tokenizer_config.json
Special tokens file saved in TimeClassification/special_tokens_map.json


# Predicción

Para poder realizar predicciones, debemos cargar el modelo que hemos entrenado anteriormente. 

In [63]:
model = BertForTokenClassification.from_pretrained("./TimeClassification")

loading configuration file ./TimeClassification/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_12": 12,
    "LABEL_13": 13,
    "LABEL_14": 14,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "

El siguiente código, permite realizar predicciones sobre una definición de PPI. Para ello, se tokeniza la entrada, se calcula la predicción sobre la misma y se realiza un postprocesado para poder mostrar correctamente el resultado. Cabe destacar que en el postprocesado, es necesario tener en cuenta que cada palabra puede estar compuesta por varias palabras tokenizadas. En este caso, decidimos que la etiqueta de la palabra compuesta sería la etiqueta de la primera palabra.

In [72]:
phrase = "average time to resolve an incident grouped by impact"
tokens  = tokenizer(phrase.split(" "), return_tensors='pt', is_split_into_words=True, truncation=True)
predictions = model(**tokens)
logits = predictions["logits"]
predictions = logits.argmax(-1).tolist()[0]

ls = [tags_list[i] for i in predictions][1:-1]

word_tag = {}
tag_list_index = 0

for word in phrase.split(" "):
    tokenized_word = tokenizer(word, return_tensors='pt', add_special_tokens=False)
    num_tokens = len(tokenized_word["input_ids"][0])
    regex =  re.search(r'^[BI]-(.*)',ls[tag_list_index])
    if regex:
        word_tag[word] = regex.group(1)
    else:
        word_tag[word] = ls[tag_list_index]
    if num_tokens == 1:
        tag_list_index += 1
    else:
        tag_list_index += num_tokens

word_tag


{'average': 'AGR',
 'time': 'O',
 'to': 'TBE',
 'resolve': 'TBE',
 'an': 'TBE',
 'incident': 'TBE',
 'grouped': 'O',
 'by': 'O',
 'impact': 'GBC'}